<a href="https://colab.research.google.com/github/Dhanraj7573/SQL-Projects/blob/main/Compound_interest_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ipywidgets pandas numpy matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def simulate_compound(
    principal: float,
    annual_return: float,
    years: float,
    contribution: float = 0.0,
    contrib_frequency: str = "monthly",  # "monthly" or "yearly"
    compounding: str = "monthly",        # "daily", "monthly", "yearly"
    inflation: float = 0.0,              # annual inflation, e.g. 0.03
    contribution_timing: str = "end",    # "start" or "end"
):
    freq_map = {"daily": 252, "monthly": 12, "yearly": 1}
    if compounding not in freq_map:
        raise ValueError("compounding must be one of: daily, monthly, yearly")
    if contrib_frequency not in ("monthly", "yearly"):
        raise ValueError("contrib_frequency must be monthly or yearly")
    if contribution_timing not in ("start", "end"):
        raise ValueError("contribution_timing must be start or end")

    periods_per_year = freq_map[compounding]
    n_periods = int(round(years * periods_per_year))

    # Convert annual return -> per-period return using geometric conversion
    r = (1.0 + annual_return) ** (1.0 / periods_per_year) - 1.0
    infl = (1.0 + inflation) ** (1.0 / periods_per_year) - 1.0

    # Contribution per period
    if contrib_frequency == "monthly":
        # If compounding isn't monthly, still approximate contributions spread across year
        contrib_per_year = contribution * 12
    else:
        contrib_per_year = contribution

    contrib_per_period = contrib_per_year / periods_per_year

    balance = float(principal)
    rows = []
    for t in range(1, n_periods + 1):
        start_bal = balance

        if contribution_timing == "start":
            balance += contrib_per_period

        interest = balance * r
        balance += interest

        if contribution_timing == "end":
            balance += contrib_per_period

        rows.append({
            "period": t,
            "year": t / periods_per_year,
            "start_balance": start_bal,
            "interest": interest,
            "contribution": contrib_per_period,
            "end_balance": balance
        })

    df = pd.DataFrame(rows)
    df["real_end_balance"] = df["end_balance"] / ((1.0 + infl) ** df["period"])
    df["total_contrib"] = df["contribution"].cumsum()
    df["total_interest"] = df["interest"].cumsum()
    return df

def time_to_goal(df: pd.DataFrame, goal: float, use_real: bool = False):
    col = "real_end_balance" if use_real else "end_balance"
    hit = df.index[df[col] >= goal]
    if len(hit) == 0:
        return None
    i = int(hit[0])
    return float(df.loc[i, "year"])

def plot_playground(df: pd.DataFrame, goal: float = None):
    plt.figure()
    plt.plot(df["year"], df["end_balance"], label="Nominal")
    plt.plot(df["year"], df["real_end_balance"], label="Real (inflation-adjusted)")
    if goal is not None:
        plt.axhline(goal)
    plt.xlabel("Years")
    plt.ylabel("Balance")
    plt.title("Compound Interest Playground")
    plt.legend()
    plt.tight_layout()
    plt.show()

# ---- Widgets UI ----
from ipywidgets import FloatSlider, Dropdown, VBox, HBox, Checkbox, interactive_output

principal_w = FloatSlider(value=10000, min=0, max=200000, step=500, description="Principal")
annual_return_w = FloatSlider(value=0.07, min=-0.2, max=0.3, step=0.005, description="Return")
years_w = FloatSlider(value=20, min=1, max=50, step=1, description="Years")
contrib_w = FloatSlider(value=300, min=0, max=5000, step=50, description="Monthly £")
compounding_w = Dropdown(options=["daily","monthly","yearly"], value="monthly", description="Compound")
inflation_w = FloatSlider(value=0.02, min=0.0, max=0.1, step=0.005, description="Inflation")
timing_w = Dropdown(options=["start","end"], value="end", description="Contrib at")
goal_w = FloatSlider(value=50000, min=0, max=500000, step=1000, description="Goal")
show_real_goal_w = Checkbox(value=False, description="Goal in real terms")

def run(principal, annual_return, years, contribution, compounding, inflation, contribution_timing, goal, show_real_goal):
    df = simulate_compound(
        principal=principal,
        annual_return=annual_return,
        years=years,
        contribution=contribution,
        contrib_frequency="monthly",
        compounding=compounding,
        inflation=inflation,
        contribution_timing=contribution_timing,
    )

    # Summary
    final_nom = df["end_balance"].iloc[-1]
    final_real = df["real_end_balance"].iloc[-1]
    total_contrib = df["total_contrib"].iloc[-1]
    total_interest = df["total_interest"].iloc[-1]

    print(f"Final (nominal): £{final_nom:,.0f}")
    print(f"Final (real):    £{final_real:,.0f}")
    print(f"Total contrib:   £{total_contrib:,.0f}")
    print(f"Total interest:  £{total_interest:,.0f}")

    # Goal timing
    yr = time_to_goal(df, goal, use_real=show_real_goal)
    if yr is None:
        print("Goal: not reached within horizon.")
    else:
        print(f"Goal reached in ~{yr:.2f} years ({'real' if show_real_goal else 'nominal'}).")

    plot_playground(df, goal=goal)

ui = VBox([
    HBox([principal_w, years_w]),
    HBox([annual_return_w, inflation_w]),
    HBox([contrib_w, compounding_w, timing_w]),
    HBox([goal_w, show_real_goal_w])
])

out = interactive_output(run, {
    "principal": principal_w,
    "annual_return": annual_return_w,
    "years": years_w,
    "contribution": contrib_w,
    "compounding": compounding_w,
    "inflation": inflation_w,
    "contribution_timing": timing_w,
    "goal": goal_w,
    "show_real_goal": show_real_goal_w
})

display(ui, out)


Output()